# Train model to detect images with an obstacle on track and make a lite version of the model

Author : Johan Jublanc
    
Date : 22/09/2019

Description : 

Use the Python interpreter to load a .tflite file and run inference

_TODO :_
* Generalize this procedure to each model in the car

## Imports

In [ ]:
import numpy as np
import tensorflow as tf

from xebikart.images import transformer as T

import xebikart.dataset as dataset
import xebikart.lite_functions as lf

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.compat.v1 import lite

In [ ]:
tf.enable_eager_execution()

# Load and convert the keras model (.h5)

The model keras (.h5) is loaded and convert to a lite model (.tflite) which is lighter :
* a smaller file thank to the serialization library FlatBuffer
* a model smaller by reducing the precision of the numbers in the model

See : 
* https://www.tensorflow.org/lite/performance/post_training_quantization
* https://www.tensorflow.org/lite/convert/index

First create a converter using the keras model as input

In [ ]:
# lite output model
model_name = "models/drive_auto_int"

# h5 intput model
run_path = "../mlruns/9/"
model_id = "ac9ca9900320450db0fc8ec3431a7859"
h5_path = "/artifacts/model/data/model.h5"
model_path = run_path + model_id + h5_path
#mlruns/8/c1a25f0c583d4d1a943e4ab270747aef/artifacts/model/data/model.h5

In [ ]:
try:
    converter = lite.TFLiteConverter.from_keras_model_file(model_path)
except:
    print("No model named {}".format(model_path))

Then we parametrize an optimizer to quantize the model so that it is smaller

In [ ]:
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

Then convert the model

In [ ]:
tflite_model = converter.convert()

And finally save the lite version of the model

In [ ]:
open("{}.tflite".format(model_name), "wb").write(tflite_model)

# Define the inference process

With tflite models, inferences are made thanks to an __interpreter__ that is fast and lean. We will define some functions to handle the whole prediction process with the Python API : 
* first, prepare the image -> read/normalize/resize/reshape/convert
* second, define an interpreter and input and ouput details
* finally, define a predictor 

In [ ]:
preprocess_fn = T.auto_drive_preprocess

In [ ]:
interpreter, input_details, output_details = lf.interpreter_and_details(model_name + ".tflite")

In [ ]:
predictor = lf.predictor_builder(model_path = model_name + ".tflite", preprocess_fn = preprocess_fn)

## Test the inference process on a random image

Get the data

In [ ]:
# dataset parameters
tubes_root_folder = "file:/workspace/xebikart-ml-tubes"
tubes_folders = [
    "tub.v4.03"
]

In [ ]:
raw_tubes_df = dataset.get_tubes_df(tubes_root_folder, tubes_folders, tubes_extension=".tar.gz")
tubes_df = raw_tubes_df.rename(columns={"cam/image_array": "images_path", "user/angle": "angles", "user/throttle": "throttles"})

Randomly select 4 images

In [ ]:
random_image_path = []
for i in range(4):
    random_image_path.append(tubes_df.sample()["images_path"].values[0])

fig, axs = plt.subplots(1, 4, figsize=(15,15), constrained_layout=True)

for i in range(4):
    # function pre-defined are used to compute the prediction
    tf_image = T.read_image(random_image_path[i])
    prediction = predictor(tf_image)
    
    # eaxh image in shown with the prediction
    axs[i].set_title("Prediction = {}".format(prediction))
    axs[i].imshow(tf_image)